In [ ]:
import os 

In [20]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from pathlib import Path
import os
import importlib
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment',  None) 


module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='preprocessing/hosp_module_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)
#print(sys.path)
root_dir = os.path.dirname(os.path.abspath('UserInterface.ipynb'))
import day_intervals_cohort
from day_intervals_cohort import *

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import data_generation

import feature_selection_hosp
from feature_selection_hosp import *

In [21]:
importlib.reload(day_intervals_cohort)
import day_intervals_cohort
from day_intervals_cohort import *

importlib.reload(day_intervals_cohort_v2)
import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

importlib.reload(data_generation_icu)
import data_generation_icu
importlib.reload(data_generation)
import data_generation
# importlib.reload(data_generation_circ)
import data_generation_circ

# importlib.reload(feature_selection_hosp)
import feature_selection_hosp
from feature_selection_hosp import *

# importlib.reload(feature_selection_icu)
import feature_selection_icu
from feature_selection_icu import *

In [ ]:
if data_icu:
    # gen=data_generation_circ.Generator(cohort_output,data_mort,data_admn,data_los,diag_flag,proc_flag,out_flag,chart_flag,med_flag,ing_flag,lab_flag,impute,include,bucket, predW)
    gen=data_generation_circ.Generator(cohort_output,data_mort,data_admn,data_los,True,True,True,True,True,True,True,impute,include,bucket, predW)
    #gen=data_generation_icu.Generator(cohort_output,data_mort,diag_flag,False,False,chart_flag,False,impute,include,bucket,predW)
    #if chart_flag:
    #    gen=data_generation_icu.Generator(cohort_output,data_mort,False,False,False,chart_flag,False,impute,include,bucket,predW)
else:
    gen=data_generation.Generator(cohort_output,data_mort,data_admn,data_los,diag_flag,lab_flag,proc_flag,med_flag,impute,include,bucket,predW)

In [18]:
def generate_adm():
    data=pd.read_csv(f"./data/cohort/cohort_icu_mortality_0_.csv.gz", compression='gzip', header=0, index_col=None)
    data['intime'] = pd.to_datetime(data['intime'])
    data['outtime'] = pd.to_datetime(data['outtime'])
    data['los']=pd.to_timedelta(data['outtime']-data['intime'],unit='h')
    data['los']=data['los'].astype(str)
    data[['days', 'dummy','hours']] = data['los'].str.split(' ', -1, expand=True)
    data[['hours','min','sec']] = data['hours'].str.split(':', -1, expand=True)
    data['los']=pd.to_numeric(data['days'])*24+pd.to_numeric(data['hours'])
    data=data.drop(columns=['days', 'dummy','hours','min','sec'])
    data=data[data['los']>0]
    data['Age']=data['Age'].astype(int)
    #print(data.head())
    #print(data.shape)
    return data

data = generate_adm()

In [19]:
data

,subject_id,stay_id,intime,outtime,Age,gender,ethnicity,insurance,label,dod,hadm_id,los
0,10000032,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,52,F,WHITE,Medicaid,0,2180-09-09 00:00:00,29079034,9
1,10000980,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,73,F,BLACK/AFRICAN AMERICAN,Medicare,0,2193-08-26 00:00:00,26913865,11
2,10001217,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,55,F,WHITE,Other,0,0,24597018,26
3,10001217,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,55,F,WHITE,Other,0,0,27703517,22
4,10001725,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,46,F,WHITE,Other,0,0,25563031,32
...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,41,M,WHITE,Medicaid,0,0,26785317,166
73177,19999625,31070865,2139-10-10 19:18:00,2139-10-11 18:21:28,81,M,WHITE,Medicare,0,0,25304202,23
73178,19999828,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,46,F,WHITE,Other,0,0,25744818,42
73179,19999840,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,58,M,WHITE,Other,1,2164-09-17 00:00:00,21033226,127


In [25]:
     
# def generate_cond():
#         cond=pd.read_csv("./data/features/preproc_diag_icu.csv.gz", compression='gzip', header=0, index_col=None)
#         cond=cond[cond['stay_id'].isin(data['stay_id'])]
#         cond_per_adm = cond.groupby('stay_id').size().max()
#         return cond, cond_per_adm
    
def generate_proc():
    proc=pd.read_csv("./data/features/preproc_proc_icu.csv.gz", compression='gzip', header=0, index_col=None)
    proc=proc[proc['stay_id'].isin(data['stay_id'])]
    proc[['start_days', 'dummy','start_hours']] = proc['event_time_from_admit'].str.split(' ', -1, expand=True)
    proc[['start_hours','min','sec']] = proc['start_hours'].str.split(':', -1, expand=True)
    proc['start_time']=pd.to_numeric(proc['start_days'])*24+pd.to_numeric(proc['start_hours'])
    proc=proc.drop(columns=['start_days', 'dummy','start_hours','min','sec'])
    proc=proc[proc['start_time']>=0]
    
    ###Remove where event time is after discharge time
    proc=pd.merge(proc,data[['stay_id','los']],on='stay_id',how='left')
    proc['sanity']=proc['los']-proc['start_time']
    proc=proc[proc['sanity']>0]
    del proc['sanity']
    
    return proc
    
def generate_out():
    out=pd.read_csv("./data/features/preproc_out_icu.csv.gz", compression='gzip', header=0, index_col=None)
    out=out[out['stay_id'].isin(data['stay_id'])]
    out[['start_days', 'dummy','start_hours']] = out['event_time_from_admit'].str.split(' ', -1, expand=True)
    out[['start_hours','min','sec']] = out['start_hours'].str.split(':', -1, expand=True)
    out['start_time']=pd.to_numeric(out['start_days'])*24+pd.to_numeric(out['start_hours'])
    out=out.drop(columns=['start_days', 'dummy','start_hours','min','sec'])
    out=out[out['start_time']>=0]
    
    ###Remove where event time is after discharge time
    out=pd.merge(out,data[['stay_id','los']],on='stay_id',how='left')
    out['sanity']=out['los']-out['start_time']
    out=out[out['sanity']>0]
    del out['sanity']
    
    return out
    
    
def generate_chart():
    chunksize = 5000000
    final=pd.DataFrame()
    for chart in tqdm(pd.read_csv("./data/features/preproc_chart_icu.csv.gz", compression='gzip', header=0, index_col=None,chunksize=chunksize)):
        chart=chart[chart['stay_id'].isin(data['stay_id'])]
        chart[['start_days', 'dummy','start_hours']] = chart['event_time_from_admit'].str.split(' ', -1, expand=True)
        chart[['start_hours','min','sec']] = chart['start_hours'].str.split(':', -1, expand=True)
        chart['start_time']=pd.to_numeric(chart['start_days'])*24+pd.to_numeric(chart['start_hours'])
        chart=chart.drop(columns=['start_days', 'dummy','start_hours','min','sec','event_time_from_admit'])
        chart=chart[chart['start_time']>=0]

        ###Remove where event time is after discharge time
        chart=pd.merge(chart,data[['stay_id','los']],on='stay_id',how='left')
        chart['sanity']=chart['los']-chart['start_time']
        chart=chart[chart['sanity']>0]
        del chart['sanity']
        del chart['los']
        
        if final.empty:
            final=chart
        else:
            final=final.append(chart, ignore_index=True)
    
    return final
    

def generate_labs():
    chunksize = 10000000
    final=pd.DataFrame()
    for labs in tqdm(pd.read_csv("./data/features/preproc_labs.csv.gz", compression='gzip', header=0, index_col=None,chunksize=chunksize)):
        labs=labs[labs['hadm_id'].isin(data['hadm_id'])]
        labs[['start_days', 'dummy','start_hours']] = labs['lab_time_from_admit'].str.split(' ', -1, expand=True)
        labs[['start_hours','min','sec']] = labs['start_hours'].str.split(':', -1, expand=True)
        labs['start_time']=pd.to_numeric(labs['start_days'])*24+pd.to_numeric(labs['start_hours'])
        labs=labs.drop(columns=['start_days', 'dummy','start_hours','min','sec'])
        labs=labs[labs['start_time']>=0]

        ###Remove where event time is after discharge time
        labs=pd.merge(labs,data[['hadm_id','los']],on='hadm_id',how='left')
        labs['sanity']=labs['los']-labs['start_time']
        labs=labs[labs['sanity']>0]
        del labs['sanity']
        
        if final.empty:
            final=labs
        else:
            final=final.append(labs, ignore_index=True)

    return final
    
    
def generate_meds():
    meds=pd.read_csv("./data/features/preproc_med_icu.csv.gz", compression='gzip', header=0, index_col=None)
    meds[['start_days', 'dummy','start_hours']] = meds['start_hours_from_admit'].str.split(' ', -1, expand=True)
    meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
    meds['start_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])
    meds[['start_days', 'dummy','start_hours']] = meds['stop_hours_from_admit'].str.split(' ', -1, expand=True)
    meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
    meds['stop_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])
    meds=meds.drop(columns=['start_days', 'dummy','start_hours','min','sec'])
    #####Sanity check
    meds['sanity']=meds['stop_time']-meds['start_time']
    meds=meds[meds['sanity']>0]
    del meds['sanity']
    #####Select hadm_id as in main file
    meds=meds[meds['stay_id'].isin(data['stay_id'])]
    meds=pd.merge(meds,data[['stay_id','los']],on='stay_id',how='left')

    #####Remove where start time is after end of visit
    meds['sanity']=meds['los']-meds['start_time']
    meds=meds[meds['sanity']>0]
    del meds['sanity']
    ####Any stop_time after end of visit is set at end of visit
    meds.loc[meds['stop_time'] > meds['los'],'stop_time']=meds.loc[meds['stop_time'] > meds['los'],'los']
    del meds['los']
    
    meds['rate']=meds['rate'].apply(pd.to_numeric, errors='coerce')
    meds['amount']=meds['amount'].apply(pd.to_numeric, errors='coerce')
    
    return meds
    
def generate_ing():
    ing=pd.read_csv("./data/features/preproc_ing_icu.csv.gz", compression='gzip', header=0, index_col=None)
    ing[['start_days', 'dummy','start_hours']] = ing['start_hours_from_admit'].str.split(' ', -1, expand=True)
    ing[['start_hours','min','sec']] = ing['start_hours'].str.split(':', -1, expand=True)
    ing['start_time']=pd.to_numeric(ing['start_days'])*24+pd.to_numeric(ing['start_hours'])
    ing[['start_days', 'dummy','start_hours']] = ing['stop_hours_from_admit'].str.split(' ', -1, expand=True)
    ing[['start_hours','min','sec']] = ing['start_hours'].str.split(':', -1, expand=True)
    ing['stop_time']=pd.to_numeric(ing['start_days'])*24+pd.to_numeric(ing['start_hours'])
    ing=ing.drop(columns=['start_days', 'dummy','start_hours','min','sec'])
    #####Sanity check
    ing['sanity']=ing['stop_time']-ing['start_time']
    ing=ing[ing['sanity']>0]
    del ing['sanity']
    #####Select hadm_id as in main file
    ing=ing[ing['stay_id'].isin(data['stay_id'])]
    ing=pd.merge(ing,data[['stay_id','los']],on='stay_id',how='left')

    #####Remove where start time is after end of visit
    ing['sanity']=ing['los']-ing['start_time']
    ing=ing[ing['sanity']>0]
    del ing['sanity']
    ####Any stop_time after end of visit is set at end of visit
    ing.loc[ing['stop_time'] > ing['los'],'stop_time']=ing.loc[ing['stop_time'] > ing['los'],'los']
    del ing['los']
    
    ing['rate']=ing['rate'].apply(pd.to_numeric, errors='coerce')
    ing['amount']=ing['amount'].apply(pd.to_numeric, errors='coerce')
    
    return ing


def get_stay_id(labs, chart):
            
    stay = pd.read_csv("./"+"mimiciv/2.2"+"/icu/icustays.csv.gz")
    stay = stay[stay.notna()]
    
    chart['charttime'] = pd.to_datetime(chart['charttime'])
    labs['charttime'] = pd.to_datetime(labs['charttime'])

    stay['intime'] = pd.to_datetime(stay['intime'])
    stay['outtime'] = pd.to_datetime(stay['outtime'])


    labs['stay_id'] = np.nan
    result = []

    unique_patient_ids = labs['subject_id'].unique()

    for p in tqdm(range(len(unique_patient_ids))):
        
        p_id = unique_patient_ids[p]
        
        lab = labs[labs['subject_id']==p_id].copy().sort_values('charttime').reset_index(drop=True)
        stay_interest = stay[stay['subject_id']==p_id].copy()
        
        unique_stay_ids = stay_interest['stay_id'].unique()
        
        for s in  tqdm(range(len(unique_stay_ids)), leave=False):
            
            stay_id = unique_stay_ids[s]
            
            stay_interest2 = stay_interest[stay_interest['stay_id']==stay_id].copy()
            
            indices = np.where((lab['charttime'].values >= stay_interest2['intime'].values) & 
                            (lab['charttime'].values <= stay_interest2['outtime'].values))

            lab['stay_id'].loc[indices[0]] = stay_id

            result.append(lab)
            
    result_df = pd.concat(result)
    labs = result_df[~(result_df['stay_id'].isnull())]
    return labs
    
    
# ing = generate_ing()

# chart = generate_chart()

# labs = generate_labs()

# labs = get_stay_id(labs, chart)

# cond, cond_per_adm = generate_cond()

proc = generate_proc()

out = generate_out()

meds = generate_meds()

In [27]:
data.to_csv('check_point_data.csv')
ing.to_csv('check_point_ing.csv')
chart.to_csv('check_point_chart.csv')
labs.to_csv('check_point_labs.csv')
# cond.to_csv('check_point_cond.csv')
proc.to_csv('check_point_proc.csv')
out.to_csv('check_point_out.csv')
meds.to_csv('check_point_meds.csv')
# cond_per_adm.to_csv('check_point_cond_per_adm.csv')

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

data = pd.read_csv('check_point_data.csv', index_col = 0)
ing = pd.read_csv('check_point_ing.csv', index_col = 0)
chart = pd.read_csv('check_point_chart.csv', index_col = 0)
labs = pd.read_csv('check_point_labs.csv', index_col = 0)
proc = pd.read_csv('check_point_proc.csv', index_col = 0)
out = pd.read_csv('check_point_out.csv', index_col = 0)
meds = pd.read_csv('check_point_meds.csv', index_col = 0)

In [2]:
include_start_time = 2*24
include_end_time = 15*24

def mortality_length(include_start_time,include_end_time,data, meds, ing, proc, out, chart, labs):
    print("include start time",include_start_time)
    print("include end time",include_end_time)
    
    data=data[(data['los'] >= include_start_time)]
    data=data[(data['los'] <= include_end_time)]
    hids=data['stay_id'].unique()
    print('num of patient: ', len(data.subject_id.unique()))
    print('num of stay: ', len(hids))

    # cond=cond[cond['stay_id'].isin(data['stay_id'])]
    
    ###MEDS

    meds=meds[meds['stay_id'].isin(data['stay_id'])]
    meds=meds[meds['start_time'] <= include_end_time]
    meds.loc[meds.stop_time > include_end_time, 'stop_time']=include_end_time
        
    ###ING

    ing=ing[ing['stay_id'].isin(data['stay_id'])]
    ing=ing[ing['start_time'] <= include_end_time]
    ing.loc[ing.stop_time > include_end_time, 'stop_time']=include_end_time
                
    
    ###PROCS

    proc=proc[proc['stay_id'].isin(data['stay_id'])]
    proc=proc[proc['start_time']<=include_end_time]
        
    ###OUT

    out=out[out['stay_id'].isin(data['stay_id'])]
    out=out[out['start_time']<=include_end_time]
        
    ###CHART

    chart=chart[chart['stay_id'].isin(data['stay_id'])]
    chart=chart[chart['start_time']<=include_end_time]
        
    ###LAB

    labs=labs[labs['stay_id'].isin(data['stay_id'])]
    labs=labs[labs['start_time']<=include_end_time]
    
    return  data, meds, ing, proc, out, chart, labs

data, meds, ing, proc, out, chart, labs = mortality_length(include_start_time,include_end_time ,data, meds, ing, proc, out, chart, labs)

include start time 48
include end time 360
num of patient:  25881
num of stay:  32811


In [3]:
final_meds=pd.DataFrame()
final_ing=pd.DataFrame()
final_proc=pd.DataFrame()
final_out=pd.DataFrame()
final_chart=pd.DataFrame()
final_labs=pd.DataFrame()


meds=meds.sort_values(by=['start_time'])
ing=ing.sort_values(by=['start_time'])
proc=proc.sort_values(by=['start_time'])
out=out.sort_values(by=['start_time'])
chart=chart.sort_values(by=['start_time'])
labs=labs.sort_values(by=['start_time'])

hids=data['stay_id'].unique()

In [4]:
sample_data = pd.concat([chart[['stay_id', 'itemid']], labs[['stay_id', 'itemid']]], axis = 0)

In [5]:
# Specify the item_ids we are interested in
required_item_ids = {225668, 50813, 220045, 220739, 223900, 223901, 223835, 220224}

# Find the stay_ids that have all the required item_ids at least once
valid_stay_ids = sample_data[sample_data['itemid'].isin(required_item_ids)].groupby('stay_id')['itemid'].nunique()
valid_stay_ids = valid_stay_ids[valid_stay_ids == len(required_item_ids)].index

In [6]:
meds = meds[meds['stay_id'].isin(valid_stay_ids)]
ing = ing[ing['stay_id'].isin(valid_stay_ids)]
proc = proc[proc['stay_id'].isin(valid_stay_ids)]
out = out[out['stay_id'].isin(valid_stay_ids)]
chart = chart[chart['stay_id'].isin(valid_stay_ids)]
labs = labs[labs['stay_id'].isin(valid_stay_ids)]
data = data[data['stay_id'].isin(valid_stay_ids)]

In [7]:
len(meds.itemid.unique())

65

In [8]:
print(len(data.subject_id.unique()))

14073


In [9]:
print(len(data.stay_id.unique()))

15884


In [10]:
data['los'].sum()

2120499

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment',  None) 

from tqdm import tqdm
import numpy as np
import pandas as pd
bucket = 1


for hid in tqdm(valid_stay_ids, desc = f'Total stay with Processing'):
    grp = data[data['stay_id']==hid]
    los = int(grp['los'].values[0])

    new_meds = meds[meds['stay_id']==hid]
    new_ing = ing[ing['stay_id']==hid]
    new_proc = proc[proc['stay_id']==hid]
    new_out = out[out['stay_id']==hid]
    new_chart = chart[chart['stay_id']==hid]
    new_labs = labs[labs['stay_id']==hid]
    
    t = 0
    for i in np.arange(0,los,bucket): 
        ###MED
        
        sub_meds=new_meds[(new_meds['start_time']>=i) & (new_meds['start_time']<i+bucket)].groupby(['stay_id','itemid','orderid']).agg({'stop_time':'max','subject_id':'max','rate':np.nanmean,'amount':np.nanmean})
        sub_meds=sub_meds.reset_index()
        sub_meds['start_time']=t
        sub_meds['stop_time']=sub_meds['stop_time']/bucket
        if final_meds.empty:
            final_meds=sub_meds
        else:
            final_meds=final_meds.append(sub_meds)

        ###ING

        sub_ing=new_ing[(new_ing['start_time']>=i) & (new_ing['start_time']<i+bucket)].groupby(['stay_id','itemid','orderid']).agg({'stop_time':'max','subject_id':'max','rate':np.nanmean,'amount':np.nanmean})
        sub_ing=sub_ing.reset_index()
        sub_ing['start_time']=t
        sub_ing['stop_time']=sub_ing['stop_time']/bucket
        if final_ing.empty:
            final_ing=sub_ing
        else:
            final_ing=final_ing.append(sub_ing)
            
        ###PROC

        sub_proc=new_proc[(new_proc['start_time']>=i) & (new_proc['start_time']<i+bucket)].groupby(['stay_id','itemid']).agg({'subject_id':'max'})
        sub_proc=sub_proc.reset_index()
        sub_proc['start_time']=t
        if final_proc.empty:
            final_proc=sub_proc
        else:    
            final_proc=final_proc.append(sub_proc)
                
        ###OUT

        sub_out=new_out[(new_out['start_time']>=i) & (new_out['start_time']<i+bucket)].groupby(['stay_id','itemid']).agg({'value':np.nanmean})
        sub_out=sub_out.reset_index()
        sub_out['start_time']=t
        if final_out.empty:
            final_out=sub_out
        else:    
            final_out=final_out.append(sub_out)
                    
                    
        ###CHART

        sub_chart=new_chart[(new_chart['start_time']>=i) & (new_chart['start_time']<i+bucket)].groupby(['stay_id','itemid']).agg({'valuenum':np.nanmean})
        sub_chart=sub_chart.reset_index()
        sub_chart['start_time']=t
        if final_chart.empty:
            final_chart=sub_chart
        else:    
            final_chart=final_chart.append(sub_chart)
                    
        ###LAB

        sub_labs=new_labs[(new_labs['start_time']>=i) & (new_labs['start_time']<i+bucket)].groupby(['stay_id','itemid']).agg({'valuenum':np.nanmean})
        sub_labs=sub_labs.reset_index()
        sub_labs['start_time']=t
        if final_labs.empty:
            final_labs=sub_labs
        else:    
            final_labs=final_labs.append(sub_labs)
        
        t=t+1

Total stay with Processing:   0%|          | 7/15884 [00:09<5:42:56,  1.30s/it]


KeyboardInterrupt: 

In [ ]:
final_meds.to_csv('check_point_meds(resampled)(resampled).csv', compression = 'gzip')
final_ing.to_csv('check_point_ing(resampled).csv', compression = 'gzip')
final_proc.to_csv('check_point_proc(resampled).csv', compression = 'gzip')
final_out.to_csv('check_point_out(resampled).csv', compression = 'gzip')
final_chart.to_csv('check_point_chart(resampled).csv', compression = 'gzip')
final_labs.to_csv('check_point_lab(resampled).csv', compression = 'gzip')

In [23]:
import os
feat_med = True
feat_ing = True
feat_proc = True
feat_out = True
feat_chart = True
impute = True
feat_lab = True

for hid in tqdm(valid_stay_ids, desc = 'Tabularize EHR for total stay 15,884'):
    grp=data[data['stay_id']==hid]
    los = int(grp['los'].values)
    
    if not os.path.exists("./data/csv/"+str(hid)):
        os.makedirs("./data/csv/"+str(hid))
    
    dyn_csv=pd.DataFrame()
    
    ###MEDS
    if(feat_med):
        feat=final_meds['itemid'].unique()
        df2=final_meds[final_meds['stay_id']==hid]
        if df2.shape[0]==0:
            amount=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            amount=amount.fillna(0)
            amount.columns=pd.MultiIndex.from_product([["MEDS"], amount.columns])
        else:
            rate=df2.pivot_table(index='start_time',columns='itemid',values='rate')
            #print(rate)
            amount=df2.pivot_table(index='start_time',columns='itemid',values='amount')
            df2=df2.pivot_table(index='start_time',columns='itemid',values='stop_time')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.ffill()
            df2=df2.fillna(0)

            rate=pd.concat([rate, add_df])
            rate=rate.sort_index()
            rate=rate.ffill()
            rate=rate.fillna(0)

            amount=pd.concat([amount, add_df])
            amount=amount.sort_index()
            amount=amount.ffill()
            amount=amount.fillna(0)
            #print(df2.head())
            df2.iloc[:,0:]=df2.iloc[:,0:].sub(df2.index,0)
            df2[df2>0]=1
            df2[df2<0]=0
            rate.iloc[:,0:]=df2.iloc[:,0:]*rate.iloc[:,0:]
            amount.iloc[:,0:]=df2.iloc[:,0:]*amount.iloc[:,0:]
          
            feat_df=pd.DataFrame(columns=list(set(feat)-set(amount.columns)))
            feat_df_rate = pd.DataFrame(columns=list(set(feat)-set(rate.columns)))

            amount=pd.concat([amount,feat_df],axis=1)
            rate=pd.concat([rate,feat_df_rate],axis=1)

            amount=amount[feat]
            amount=amount.fillna(0)
            rate=rate[feat]
            rate=rate.fillna(0)
#                 print(amount.columns)
            amount.columns=pd.MultiIndex.from_product([["MEDS"], amount.columns])
            rate.columns=pd.MultiIndex.from_product([["MEDS Rate"], rate.columns])

            medication = pd.concat([amount, rate], axis = 1)
            
        if(dyn_csv.empty):
            dyn_csv=medication
        else:
            dyn_csv=pd.concat([dyn_csv,medication],axis=1)
        
    
    ###INGS
    if(feat_ing):
        feat=final_ing['itemid'].unique()
        df2=final_ing[final_ing['stay_id']==hid]
        if df2.shape[0]==0:
            amount=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            amount=amount.fillna(0)
            amount.columns=pd.MultiIndex.from_product([["INGS"], amount.columns])
        else:
            rate=df2.pivot_table(index='start_time',columns='itemid',values='rate')
            #print(rate)
            amount=df2.pivot_table(index='start_time',columns='itemid',values='amount')
            df2=df2.pivot_table(index='start_time',columns='itemid',values='stop_time')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.ffill()
            df2=df2.fillna(0)

            rate=pd.concat([rate, add_df])
            rate=rate.sort_index()
            rate=rate.ffill()
            rate=rate.fillna(0)

            amount=pd.concat([amount, add_df])
            amount=amount.sort_index()
            amount=amount.ffill()
            amount=amount.fillna(0)
            
            df2.iloc[:,0:]=df2.iloc[:,0:].sub(df2.index,0)
            df2[df2>0]=1
            df2[df2<0]=0
            rate.iloc[:,0:]=df2.iloc[:,0:]*rate.iloc[:,0:]
            amount.iloc[:,0:]=df2.iloc[:,0:]*amount.iloc[:,0:]

            feat_df=pd.DataFrame(columns=list(set(feat)-set(amount.columns)))
            feat_df_rate = pd.DataFrame(columns=list(set(feat)-set(rate.columns)))

            amount=pd.concat([amount,feat_df],axis=1)
            rate=pd.concat([rate,feat_df_rate],axis=1)

            amount=amount[feat]
            amount=amount.fillna(0)
            rate=rate[feat]
            rate=rate.fillna(0)
            
            amount.columns=pd.MultiIndex.from_product([["INGS"], amount.columns])
            rate.columns=pd.MultiIndex.from_product([["INGS Rate"], rate.columns])

            ingredients = pd.concat([amount, rate], axis = 1)
            
        if(dyn_csv.empty):
            dyn_csv=medication
        else:
            dyn_csv=pd.concat([dyn_csv,medication],axis=1)
        
        
    
    
    ###PROCS
    if(feat_proc):
        feat=final_proc['itemid'].unique()
        df2=final_proc[final_proc['stay_id']==hid]
        if df2.shape[0]==0:
            df2=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            df2=df2.fillna(0)
            df2.columns=pd.MultiIndex.from_product([["PROC"], df2.columns])
        else:
            df2['val']=1
            #print(df2)
            df2=df2.pivot_table(index='start_time',columns='itemid',values='val')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.fillna(0)
            df2[df2>0]=1

            feat_df=pd.DataFrame(columns=list(set(feat)-set(df2.columns)))
            df2=pd.concat([df2,feat_df],axis=1)

            df2=df2[feat]
            df2=df2.fillna(0)
            df2.columns=pd.MultiIndex.from_product([["PROC"], df2.columns])
        
        if(dyn_csv.empty):
            dyn_csv=df2
        else:
            dyn_csv=pd.concat([dyn_csv,df2],axis=1)
        
        
    ###OUT
    if(feat_out):
        feat=final_out['itemid'].unique()
        df2=final_out[final_out['stay_id']==hid]
    
        if df2.shape[0]==0:
            val=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["OUT"], val.columns])
        else:
            val=df2.pivot_table(index='start_time',columns='itemid',values='value')
            df2['val']=1
            df2=df2.pivot_table(index='start_time',columns='itemid',values='val')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.fillna(0)

            val=pd.concat([val, add_df])
            val=val.sort_index()
            val=val.fillna(0)
            
            df2[df2>0]=1
            df2[df2<0]=0

            feat_df=pd.DataFrame(columns=list(set(feat)-set(val.columns)))
            val=pd.concat([val,feat_df],axis=1)

            val=val[feat]
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["OUT"], val.columns])
        
        if(dyn_csv.empty):
            dyn_csv=val
        else:
            dyn_csv=pd.concat([dyn_csv,val],axis=1)
            
        
    ###CHART
    if(feat_chart):
        feat=final_chart['itemid'].unique()
        df2=final_chart[final_chart['stay_id']==hid]
        if df2.shape[0]==0:
            val=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["CHART"], val.columns])
        else:
            val=df2.pivot_table(index='start_time',columns='itemid',values='valuenum')
            df2['val']=1
            df2=df2.pivot_table(index='start_time',columns='itemid',values='val')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.fillna(0)

            val=pd.concat([val, add_df])
            val=val.sort_index()
            if impute:
                val=val.ffill()
                val=val.bfill()
                val=val.interpolate()
            val=val.fillna(0)

            df2[df2>0]=1
            df2[df2<0]=0

            feat_df=pd.DataFrame(columns=list(set(feat)-set(val.columns)))
            val=pd.concat([val,feat_df],axis=1)

            val=val[feat]
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["CHART"], val.columns])
        
        if(dyn_csv.empty):
            dyn_csv=val
        else:
            dyn_csv=pd.concat([dyn_csv,val],axis=1)
    
    ###LABS
    if(feat_lab):
        feat=final_labs['itemid'].unique()
        df2=final_labs[final_labs['stay_id']==hid]
        if df2.shape[0]==0:
            val=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["LAB"], val.columns])
        else:
            val=df2.pivot_table(index='start_time',columns='itemid',values='valuenum')
            df2['val']=1
            df2=df2.pivot_table(index='start_time',columns='itemid',values='val')
            #print(df2.shape)
            add_indices = pd.Index(range(los)).difference(df2.index)
            add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
            df2=pd.concat([df2, add_df])
            df2=df2.sort_index()
            df2=df2.fillna(0)

            val=pd.concat([val, add_df])
            val=val.sort_index()
            if impute:
                val=val.ffill()
                val=val.bfill()
                val=val.fillna(val.mean())
            val=val.fillna(0)

            df2[df2>0]=1
            df2[df2<0]=0
            
            feat_df=pd.DataFrame(columns=list(set(feat)-set(val.columns)))
            val=pd.concat([val,feat_df],axis=1)

            val=val[feat]
            val=val.fillna(0)
            val.columns=pd.MultiIndex.from_product([["LAB"], val.columns])
        
        if(dyn_csv.empty):
            dyn_csv=val
        else:
            dyn_csv=pd.concat([dyn_csv,val],axis=1)
    
    #Save temporal data to csv
    dyn_csv.to_csv('./data/csv/'+str(hid)+'/dynamic.csv',index=False)

Tabularize EHR for total stay 15,884:  43%|████▎     | 6807/15884 [01:25<01:54, 79.20it/s] 


KeyboardInterrupt: 